##### Copyright 2019 The TensorFlow Authors.



In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TinyImageNet Custom ResNet


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/template/notebook.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/template/notebook.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview
{TODO: Fill in detailed info of what this accomplishes}

## Setup

In [0]:
! pip install -q tensorflow-gpu==2.0.0-beta1
import tensorflow as tf
assert tf.__version__.startswith('2')

print(f'{tf.__version__}')

In [0]:
! git clone https://github.com/Vishal-V/GSoC.git

In [0]:
from load_data import *

## Download the data

In [0]:
import os
download_path = os.getcwd()
    
import pathlib
path = tf.keras.utils.get_file('tiny-imagenet-200.zip', extract=True, cache_subdir=download_path,
                               origin='http://cs231n.stanford.edu/tiny-imagenet-200.zip')

data_dir = pathlib.Path(path).with_suffix('')

TRAIN = data_dir/"train"
VAL = data_dir/"val/images"
VAL_ANNOT = data_dir/'val/val_annotations.txt'

## Image augmentation and image generators
- The function below returns the generators for the ImageDataGenerator objects we will use to train and validate our ResNet model.


In [0]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

val_data = pd.read_csv(VAL_ANNOT , sep='\t', names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X','Y','H', 'W'], axis=1, inplace=True)


def train_val_gen(train_target=64, train_batch=64, val_target=64, val_batch=64):
		
		train_datagen = ImageDataGenerator(
		        rescale=1./255,
		        rotation_range=18, # Rotation Angle
		        zoom_range=0.15,  # Zoom Range
		        width_shift_range=0.2, # Width Shift
		        height_shift_range=0.2, # Height Shift
		        shear_range=0.15,  # Shear Range
		        horizontal_flip=True, # Horizontal Flip
		        fill_mode="reflect", # Fills empty with reflections
		        brightness_range=[0.4, 1.6]  # Increasing/decreasing brightness
		)

		train_generator = train_datagen.flow_from_directory(
		        TRAIN,
		        target_size=(train_target, train_target),
		        batch_size=train_batch,
		        class_mode='categorical')

		val_datagen = ImageDataGenerator(rescale=1./255)

		val_generator = val_datagen.flow_from_dataframe(
		    val_data, directory=VAL, 
		    x_col='File', 
		    y_col='Class', 
		    target_size=(val_target, val_target),
		    color_mode='rgb', 
		    class_mode='categorical', 
		    batch_size=val_batch, 
		    shuffle=False, 
		    seed=42
		)

		return train_generator, val_generator

## Defining callbacks to employ different training strategies

## Custom ResNet that uses Pre-Activation and BottleNeck Blocks with SeparableConv2D

---


-  We use 1x1 to increase the number of channels is to create a wider model with minimum increase in trainable parameters
- This [reserach paper](https://arxiv.org/abs/1812.01187) documents improved accuracy with AveragePooling2D in the shortcut connection. This model showed a performance drop and hence was replaced with a 1x1 convolution.
- Uses SeparableConv2D rather than vanilla Conv2D to reduce the nmber of parameters and make the model feasible to train on constrained environments like colab.

In [0]:
from tensorflow.keras.layers import BatchNormalization, Conv2D, SeparableConv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import ZeroPadding2D, Activation, Flatten, add
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [0]:
class ResNet:
    
    def residual_block(data, K, stride, chanDim, red=False, reg=0.0001, bnEps=2e-5, bnMom=0.9):
        shortcut = data

        bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(data)
        act1 = Activation("relu")(bn1)
        conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False, kernel_regularizer=l2(reg))(act1)

        bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv1)
        act2 = Activation("relu")(bn2)
        conv2 = Conv2D(int(K * 0.25), (3, 3), strides=stride, padding="same", use_bias=False, kernel_regularizer=l2(reg))(act2)

        bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(conv2)
        act3 = Activation("relu")(bn3)
        conv3 = Conv2D(K, (1, 1), use_bias=False, kernel_regularizer=l2(reg))(act3)

        if red:
            shortcut = Conv2D(K, (1, 1), strides=stride, use_bias=False, kernel_regularizer=l2(reg))(act1)

        x = add([conv3, shortcut])

        return x

    def build(width, height, depth, classes, stages, filters, reg=0.0001, bnEps=2e-5, bnMom=0.9):
        inputShape = (height, width, depth)
        chanDim = -1

        inputs = tf.keras.Input(shape=inputShape)
        x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(inputs)

        x = Conv2D(filters[0], (5, 5), use_bias=False, padding="same", kernel_regularizer=l2(reg))(x)
        x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
        x = Activation("relu")(x)
        x = ZeroPadding2D((1, 1))(x)
        x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    
        for i in range(0, len(stages)):
            stride = (1, 1) if i == 0 else (2, 2)
            x = ResNet.residual_block(x, filters[i + 1], stride, chanDim, red=True, bnEps=bnEps, bnMom=bnMom)

            for j in range(0, stages[i] - 1):
                x = ResNet.residual_block(x, filters[i + 1], (1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)

        x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
        x = Activation("relu")(x)
        x = AveragePooling2D((8, 8))(x)
        x = Conv2D(200, (1,1), kernel_regularizer=l2(reg))(x)
        x = Flatten()(x)
        x = Activation("softmax")(x)

        model = tf.keras.Model(inputs, x, name="resnet")

        return model

In [0]:
model = ResNet.build(64, 64, 3, 200, (3, 4, 6), (64, 128, 256, 512), reg=0.0005)

In [0]:
model.summary()

## Compile the model

In [0]:
opt = Adam(learning_rate=0.1, beta_1=0.9, beta_2=0.999, epsilon=0.1, amsgrad=False)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

## Using fit_generator to train the model
ImageDataGenerator is best suited for augmenting images on the fly and training the model

In [0]:
train_gen, val_gen = train_val_gen(train_target=64, train_batch=64, val_target=64, val_batch=64)

In [0]:
model.fit_generator(
  train_gen,
  steps_per_epoch=100000 // 64,
  validation_data=val_gen,
  validation_steps=10000 // 64,
  epochs=20,
  max_queue_size=64,
  verbose=1
)

### List of references for easy lookup

---

1. Building blocks of interpretability: [Link](https://distill.pub/2018/building-blocks/) (Holy Grail of Intuition!)
2. Deep Residual Learning for image classification: [Link](https://arxiv.org/abs/1512.03385) (Resnet Paper)
3. Bag of tricks for image classification: [Link](https://arxiv.org/abs/1812.01187) (Tweaks and tricks to Resnet for increased performance paper)
2. Imbalanced Deep Learning by Minority Class
Incremental Rectification: [Link](https://arxiv.org/pdf/1804.10851.pdf) (Selectively Sampling Data paper)
2. Improved Regularization of Convolutional Neural Networks with Cutout: [Link](https://arxiv.org/pdf/1708.04552.pdf) (Cutout/Occlusion Augmentation paper)
3. Survey of resampling techniques for improving
classification performance in unbalanced datasets [Link](https://arxiv.org/pdf/1608.06048v1.pdf) (Resampling paper)

## Notes

For general instructions on how to write docs for Tensorflow see [Writing TensorFlow Documentation](https://www.tensorflow.org/community/documentation).

The tips below are specific to notebooks for tensorflow.

### General

* Include the collapsed license at the top (this uses Colab's "Form" mode to hide the cells).
* Only include a single `H1` title.
* Include the button-bar immediately under the `H1`.
* Include an overview section before any code.
* Put all your installs and imports in a setup section.
* Always include the three `__future__` imports.
* Save the notebook with the Table of Contents open.
* Write python3 compatible code.
* Keep cells small (~max 20 lines).


### Working in GitHub

* Be consistent about how you save your notebooks, otherwise the JSON-diffs will be a mess.

* This notebook has the "Omit code cell output when saving this notebook" option set. GitHub refuses to diff notebooks with large diffs (inline images).

* [reviewnb.com](http://reviewnb.com) may help. You can access it using this bookmarklet:

  ```
javascript:(function(){ window.open(window.location.toString().replace(/github\.com/, 'app.reviewnb.com').replace(/files$/,"")); })()
 ```
 
* To open a GitHub notebook in Colab use the [Open in Colab](https://chrome.google.com/webstore/detail/open-in-colab/iogfkhleblhcpcekbiedikdehleodpjo) extension (or make a bookmarklet).
  
* The easiest way to edit a notebook in GitHub is to open it with Colab from the branch you want to edit. Then use File --> Save a copy in GitHub, which will save it back to the branch you opened it from.

* For PRs it's helpful to post a direct Colab link to the PR head: https://colab.research.google.com/github/{user}/{repo}/blob/{branch}/{path}.ipynb


### Code Style


* Notebooks are for people. Write code optimized for clarity.

* Demonstrate small parts before combining them into something more complex. Like below:

In [0]:
#Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(None, 5)),
    tf.keras.layers.Dense(3)
])

In [0]:
# Run the model on a single batch of data, and inspect the output.
result = model(tf.constant(np.random.randn(10,5), dtype = tf.float32)).numpy()

print("min:", result.min())
print("max:", result.max())
print("mean:", result.mean())
print("shape:", result.shape)

In [0]:
# Compile the model for training
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.categorical_crossentropy)

* Keep examples quick. Use small datasets, or small slices of datasets. You don't need to train to convergence, train until it's obvious it's making progress.

* For a large example, don't try to fit all the code in the notebook. Add python files to tensorflow examples, and in the noptebook run: `!pip install git+https://github.com/tensorflow/examples`

### Code content

Use the highest level API that gets the job done (unless the goal is to demonstrate the low level API).

Use `keras.Sequential` > keras functional api > keras model subclassing > ...

Use  `model.fit` > `model.train_on_batch` > manual `GradientTapes`.

Use eager-style code.

Use `tensorflow_datasets` and `tf.data` where possible.

Avoid `compat.v1`.



### Text

* Use an imperative style. "Run a batch of images through the model."

* Use sentence case in titles/headings. 

* Use short titles/headings: "Download the data", "Build the Model", "Train the model".

